In [ ]:
import torch
import numpy as np
from PIL import Image, ImageSequence

from diffusion_wrapper import DiffusionWrapper
from heatmap_generator import HeatmapGenerator
from zero_shot_tracker import ZeroShotTracker


diffusion_wrapper = DiffusionWrapper('../text-to-video-ms-1.7b')
heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

video_features_dict = diffusion_wrapper.extract_video_features('../rocket256.gif', "A rocket starting on Mars.")

In [ ]:
video_features = diffusion_wrapper.concatenate_video_features({'up_block': [video_features_dict['up_block'][2]]})
video_features = video_features.permute(0, 2, 3, 1).float().cpu()

In [ ]:
heatmaps = heatmap_generator.generate(video_features, (93, 137, 0))
heatmap_generator.safe_heatmap_as_gif(heatmaps)

#heatmaps = torch.permute(heatmaps, (0, 3, 1, 2))
#heatmaps = torch.nn.functional.interpolate(heatmaps, size=256, mode="bilinear", align_corners=True) * 255
#
#heatmaps = heatmaps.squeeze().numpy()
#frames_gif = [Image.fromarray(f) for f in heatmaps]
#frames_gif[0].save("output/heatmaps.gif", save_all=True, append_images=frames_gif[1:], duration=100, loop=0)

tracks = zero_shot_tracker.track(heatmaps)

def load_frames(image: Image, mode='RGB'):
    return np.array([
        np.array(frame.convert(mode))
        for frame in ImageSequence.Iterator(image)
    ])

with Image.open('../rocket256.gif') as im:
    frames = load_frames(im)

zero_shot_tracker.place_marker_in_frames(frames, tracks)